## Multi-LLM Integrations

This notebook involves integrating multiple LLMs, a way to get comfortable working with LLM APIs.
I'll be using Amazon Bedrock, which has a number of models that can be accessed via AWS SDK Boto3 library. I'll also use Deepseek directly via the API.

In [ ]:
# Importing required libraries
# Boto3 library is AWS SDK for Python providing the necessary set of libraries (uv pip install boto3)

import os
import json
import boto3
from openai import OpenAI
from dotenv import load_dotenv
from IPython.display import Markdown, display

In [ ]:
# Always remember to do this!
load_dotenv(override=True)

In [ ]:
# Print the key prefixes to help with any debugging

amazon_bedrock_bedrock_api_key = os.getenv('AMAZON_BEDROCK_API_KEY')
deepseek_api_key = os.getenv('DEEPSEEK_API_KEY')

if amazon_bedrock_bedrock_api_key:
    print(f"Amazon Bedrock API Key exists and begins {amazon_bedrock_bedrock_api_key[:4]}")
else:
    print("Amazon Bedrock API Key not set (and this is optional)")

if deepseek_api_key:
    print(f"DeepSeek API Key exists and begins {deepseek_api_key[:3]}")
else:
    print("DeepSeek API Key not set (and this is optional)")

In [ ]:
# Amazon Bedrock Client

bedrock_client = boto3.client(
    service_name="bedrock-runtime",
    region_name="us-east-1"
)

# Deepseek Client

deepseek_client = OpenAI(
    api_key=deepseek_api_key, 
    base_url="https://api.deepseek.com"
)

In [ ]:
# Coming up with message for LLM Evaluation
text = "Please come up with a challenging, nuanced question that I can ask a number of LLMs to evaluate their intelligence. "
text += "Answer only with the question, no explanation."
messages = [{"role": "user", "content": [{"text": text}]}]

In [ ]:
messages

In [ ]:
# Anthropic Claude 3.5 Sonnet for model evaluator question

model_id = "anthropic.claude-3-5-sonnet-20240620-v1:0"
response = bedrock_client.converse(
    modelId=model_id,
    messages=messages,
)
model_evaluator_question = response['output']['message']['content'][0]['text']
print(model_evaluator_question)

In [ ]:
competitors = []
answers = []
messages = [{"role": "user", "content": model_evaluator_question}]

In [ ]:
# Deepseek chat model answer

model_id = "deepseek-chat"
response = deepseek_client.chat.completions.create(
    model=model_id,
    messages=messages
)
answer = response.choices[0].message.content

display(Markdown(answer))
competitors.append(model_id)
answers.append(answer)

In [ ]:
messages = [{"role": "user", "content": [{"text": model_evaluator_question}]}]
messages

In [ ]:
# Amazon nova lite

model_id = "amazon.nova-lite-v1:0"
response = bedrock_client.converse(
    modelId=model_id,
    messages=messages,
)
answer = response['output']['message']['content'][0]['text']

display(Markdown(answer))
competitors.append(model_id)
answers.append(answer)

In [ ]:
# Amazon Nova Pro

model_id = "amazon.nova-pro-v1:0"
response = bedrock_client.converse(
    modelId=model_id,
    messages=messages,
)
answer = response['output']['message']['content'][0]['text']

display(Markdown(answer))
competitors.append(model_id)
answers.append(answer)

In [ ]:
messages = [{"role": "user", "content": [{"text": model_evaluator_question}]}]
messages

In [ ]:
# Cohere Command Light

model_id = "cohere.command-light-text-v14"
response = bedrock_client.converse(
    modelId=model_id,
    messages=messages,
)
answer = response['output']['message']['content'][0]['text']

display(Markdown(answer))
competitors.append(model_id)
answers.append(answer)

## For the next cell, we will use Ollama

Ollama runs a local web service that gives an OpenAI compatible endpoint,  
and runs models locally using high performance C++ code.

If you don't have Ollama, install it here by visiting https://ollama.com then pressing Download and following the instructions.

After it's installed, you should be able to visit here: http://localhost:11434 and see the message "Ollama is running"

You might need to restart Cursor (and maybe reboot). Then open a Terminal (control+\`) and run `ollama serve`

Useful Ollama commands (run these in the terminal, or with an exclamation mark in this notebook):

`ollama pull <model_id>` downloads a model locally    
`ollama ls` lists all the models you've downloaded  
`ollama rm <model_id>` deletes the specified model from your downloads   
`ollama run <model_id>` pulls the model if it doesn't exist locally, and run it.

<table style="margin: 0; text-align: left; width:100%">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../../../assets/stop.png" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#ff7800;">Important</h2>
            <span style="color:#ff7800;">The model called <b>llama3.3</b> is FAR too large for home computers - it's not intended for personal computing and will consume all your resources! Stick with the nicely sized <b>llama3.2</b> or <b>llama3.2:1b</b> and if you want larger, try llama3.1 or smaller variants of Qwen, Gemma, Phi or DeepSeek. See the <A href="https://ollama.com/models">the Ollama models page</a> for a full list of models and sizes.
            </span>
        </td>
    </tr>
</table>

In [ ]:
!ollama run llama3.2:1b

In [ ]:
messages = [{"role": "user", "content": model_evaluator_question}]
messages

In [ ]:
ollama = OpenAI(base_url='http://localhost:11434/v1', api_key='ollama')
model_id = "llama3.2:1b"

response = ollama.chat.completions.create(
    model=model_id, 
    messages=messages
)
answer = response.choices[0].message.content

display(Markdown(answer))
competitors.append(model_id)
answers.append(answer)

In [ ]:
# Listing all models and their answers
print(competitors)
print(answers)

In [ ]:
# Mapping each model with it's solution for the model evaluator question
for competitor, answer in zip(competitors, answers):
    print(f"Competitor: {competitor}\n\n{answer}")

In [ ]:
# Masking out the model name for evaluation purposes - note the use of "enumerate"

together = ""
for index, answer in enumerate(answers):
    together += f"# Response from competitor {index+1}\n\n"
    together += answer + "\n\n"

In [ ]:
print(together)

In [ ]:
judge = f"""You are judging a competition between {len(competitors)} competitors.
Each model has been given this question:

{model_evaluator_question}

Your job is to evaluate each response for clarity and strength of argument, and rank them in order of best to worst.
Respond with JSON, and only JSON, with the following format:
{{"results": ["best competitor number", "second best competitor number", "third best competitor number", ...]}}

Here are the responses from each competitor:

{together}

Now respond with the JSON with the ranked order of the competitors, nothing else. Do not include markdown formatting or code blocks."""


In [ ]:
print(judge)

In [ ]:
judge_messages = [{"role": "user", "content": [{"text": judge}]}]
judge_messages

In [ ]:
# Anthropic Claude 3.5 Sonnet for model evaluator question

model_id = "anthropic.claude-3-5-sonnet-20240620-v1:0"
response = bedrock_client.converse(
    modelId=model_id,
    messages=judge_messages,
)
model_evaluator_response = response['output']['message']['content'][0]['text']
print(model_evaluator_response)

In [ ]:
# OK let's turn this into results!

results_dict = json.loads(model_evaluator_response)
ranks = results_dict["results"]
for index, result in enumerate(ranks):
    competitor = competitors[int(result)-1]
    print(f"Rank {index+1}: {competitor}")

<table style="margin: 0; text-align: left; width:100%">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../../../assets/business.png" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#00bfff;">Commercial implications</h2>
            <span style="color:#00bfff;">These kinds of patterns - to send a task to multiple models, and evaluate results,
            are common where you need to improve the quality of your LLM response. This approach can be universally applied
            to business projects where accuracy is critical.
            </span>
        </td>
    </tr>
</table>